In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Bidirectional, LSTM, Dropout,Input, Embedding,GRU
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, AUC, binary_accuracy, TrueNegatives, TruePositives, FalseNegatives, FalsePositives
from helper_functions import smi_tokenizer, mcc_metric
from rdkit import Chem
from rdkit.Chem import PandasTools
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

In [2]:
data = pd.read_table('F:/bbb/data/B3DB/B3DB_classification.tsv')
data.head(1)

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN


In [3]:
X = data["SMILES"]

target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [4]:
smile_data = [smi_tokenizer(smile) for smile in X]

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(smile_data, y_classification, test_size=0.2, random_state=42)
xtrain, xvalid, ytrain, yvalid  = train_test_split(xtrain, ytrain, test_size= 0.16, random_state=42)

In [6]:
len(y_classification)

7807

In [7]:
np.unique(y_classification, return_counts=True)

(array([0, 1], dtype=int64), array([4956, 2851], dtype=int64))

In [8]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(ytrain), 
            ytrain)

F:\anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 0 ... 0 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [9]:
class_weights

array([0.79038577, 1.36092372])

In [10]:
class_weights = {0:0.78950695,
                1: 1.36353712} 

In [11]:
text_vectorizer = TextVectorization(max_tokens=None, standardize=None, output_sequence_length=None,)

In [12]:
text_vectorizer.adapt(smile_data)
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab)

67

In [13]:
input = Input(shape = (1,), dtype = 'string')
x = text_vectorizer(input)
x = Embedding(input_dim=len(word_in_vocab),
                        output_dim=128, 
                        input_length=None,
                        )(x)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(64, return_sequences=False,name = 'lstm1'))(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
output = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = output)


In [14]:
def save(i):
    return 'F:/bbb/model/nlp/model_'+str(i)+'.tf'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save(33), 
                monitor='val_auc', verbose=1, 
                save_best_only=True, save_weights_only = False, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001) ,loss = BinaryCrossentropy(), metrics = [binary_accuracy, AUC(), mcc_metric 
                    ,Precision(),TrueNegatives(), TruePositives(), FalseNegatives(), FalsePositives()])
model.fit(x = np.array(xtrain), y = np.array(ytrain), validation_data = (np.array(xvalid), np.array(yvalid)), epochs=60, batch_size = 32, 
              callbacks = callbacks_list, class_weight = class_weights)


Epoch 1/60
164/164 [==============================] - ETA: 0s - loss: 0.5587 - binary_accuracy: 0.6976 - auc: 0.7753 - mcc_metric: nan - precision: 0.5721 - true_negatives: 2306.0000 - true_positives: 1353.0000 - false_negatives: 574.0000 - false_positives: 1012.0000
Epoch 00001: val_auc improved from -inf to 0.84196, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 74s 323ms/step - loss: 0.5587 - binary_accuracy: 0.6976 - auc: 0.7753 - mcc_metric: nan - precision: 0.5721 - true_negatives: 2306.0000 - true_positives: 1353.0000 - false_negatives: 574.0000 - false_positives: 1012.0000 - val_loss: 0.6264 - val_binary_accuracy: 0.7460 - val_auc: 0.8420 - val_mcc_metric: 0.4876 - val_precision: 0.6172 - val_true_negatives: 459.0000 - val_true_positives: 287.0000 - val_false_negatives: 76.0000 - val_false_positives: 178.0000
Epoch 2/60
164/164 [==============================] - ETA: 0s - loss: 0.5065 - binary_accuracy: 0.7847 - auc: 0.8184 - mcc_metric: 0.5457 - precision: 0.7127 - true_negatives: 2779.0000 - true_positives: 1337.0000 - false_negatives: 590.0000 - false_positives: 539.0000
Epoch 00002: val_auc improved from 0.84196 to 0.84921, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


...

INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 44s 268ms/step - loss: 0.1609 - binary_accuracy: 0.9325 - auc: 0.9833 - mcc_metric: 0.8599 - precision: 0.8758 - true_negatives: 3058.0000 - true_positives: 1833.0000 - false_negatives: 94.0000 - false_positives: 260.0000 - val_loss: 0.3720 - val_binary_accuracy: 0.8800 - val_auc: 0.9359 - val_mcc_metric: 0.7471 - val_precision: 0.8275 - val_true_negatives: 573.0000 - val_true_positives: 307.0000 - val_false_negatives: 56.0000 - val_false_positives: 64.0000
Epoch 35/60
164/164 [==============================] - ETA: 0s - loss: 0.1462 - binary_accuracy: 0.9386 - auc: 0.9865 - mcc_metric: 0.8698 - precision: 0.8898 - true_negatives: 3091.0000 - true_positives: 1832.0000 - false_negatives: 95.0000 - false_positives: 227.0000
Epoch 00035: val_auc improved from 0.93590 to 0.93916, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 49s 300ms/step - loss: 0.1462 - binary_accuracy: 0.9386 - auc: 0.9865 - mcc_metric: 0.8698 - precision: 0.8898 - true_negatives: 3091.0000 - true_positives: 1832.0000 - false_negatives: 95.0000 - false_positives: 227.0000 - val_loss: 0.3782 - val_binary_accuracy: 0.8700 - val_auc: 0.9392 - val_mcc_metric: 0.7308 - val_precision: 0.7935 - val_true_negatives: 555.0000 - val_true_positives: 315.0000 - val_false_negatives: 48.0000 - val_false_positives: 82.0000
Epoch 36/60
164/164 [==============================] - ETA: 0s - loss: 0.1369 - binary_accuracy: 0.9417 - auc: 0.9874 - mcc_metric: 0.8789 - precision: 0.8876 - true_negatives: 3083.0000 - true_positives: 1856.0000 - false_negatives: 71.0000 - false_positives: 235.0000
Epoch 00036: val_auc did not improve from 0.93916
164/164 [==============================] - 11s 70ms/step - loss: 0.1369 - binary_accuracy: 0.9417 - auc: 0.9874 - mcc_metric: 0.8789 - precision: 0.8876 - true_negatives: 3083

In [15]:
model.load_weights(save(33))
model.evaluate(np.array(xtest), np.array(ytest))

49/49 [==============================] - 1s 27ms/step - loss: 0.4953 - binary_accuracy: 0.8406 - auc: 0.9156 - mcc_metric: 0.6660 - precision: 0.7393 - true_negatives: 831.0000 - true_positives: 482.0000 - false_negatives: 79.0000 - false_positives: 170.0000


[0.4952695071697235,
 0.8405889868736267,
 0.9156013131141663,
 0.6659770011901855,
 0.7392638325691223,
 831.0,
 482.0,
 79.0,
 170.0]

In [16]:
model.evaluate(np.array(xvalid), np.array(yvalid))

32/32 [==============================] - 1s 32ms/step - loss: 0.3782 - binary_accuracy: 0.8700 - auc: 0.9392 - mcc_metric: 0.7308 - precision: 0.7935 - true_negatives: 555.0000 - true_positives: 315.0000 - false_negatives: 48.0000 - false_positives: 82.0000


[0.3782069683074951,
 0.8700000047683716,
 0.9391647577285767,
 0.7307599186897278,
 0.7934508919715881,
 555.0,
 315.0,
 48.0,
 82.0]

In [17]:
tf.__version__

'2.7.0'